In [1]:
"""
Created on Wed Oct 19 15:48:00 2018

@author: Daniel Cuesta, Alejandro Garo
"""

import string, random
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import math
import cmath

from cmath import pi, exp

In [2]:
t =np.array([1,2,1,0])

In [3]:
def fft(t):
    
    k = len(t)
    
    if k == 1 :
        return t
    
    pow2 = math.ceil(math.log(k,2))
    # print(pow2)
    
    t = np.array(list(t) + (2**pow2 - k) * [float(0)], dtype = complex)
    # t.append(np.zeros(2**log2K-k))
    
    # print(t)
        
    c_e = t[::2]
    c_o = t[1::2]
    
    f_e = fft(c_e)
    f_o = fft(c_o)
    # print("f_e ",f_e)
    # print("f_o ",f_o)
    for i in range((2**pow2)-1):
        t[i] = f_e[i % 2**(pow2-1)] + exp(2j*pi*(i/2**pow2)) * f_o[i % 2**(pow2-1)]
    return t

In [4]:
def invert_fft(t, fft_func=fft):
    k = len(t)
    
    # if k == 1 :
    #     return t
    
    pow2 = math.ceil(math.log(k,2))
    # print(pow2)
    
    t = np.array(list(t) + (2**pow2 - k) * [float(0)], dtype = complex)
    # t.append(np.zeros(2**log2K-k))
    
    # print(t)
    
    return fft_func(t)
    

In [5]:
# Definimos t
t =np.array([1,2,1,0])

# Aplicamos la fft sobre el
# t_fft = fft(t)
# print(t)
# print(t_fft)

# FFT de numpy
print(t)
t_fft = np.fft.fft(t)
print(t_fft)

# t_inv_fft = invert_fft(t_fft, fft_func = fft) 
# print (t_inv_fft)

t_inv_fft2 = invert_fft(t_fft, fft_func = np.fft.fft)
print(t_inv_fft2)


[1 2 1 0]
[4.+0.j 0.-2.j 0.+0.j 0.+2.j]
[4.+0.j 0.+0.j 4.+0.j 8.+0.j]


In [6]:
# 1-B: Multiplicando polinomios y numeros

# Genera un polinomio aleatorio
def rand_polinomio(long = 2**10, base = 10):
    # Comprobacion de la base
    if base < 2 or base > 10:
        return -1
    
    # Computo del polinomio
    l_pol = np.random.randint(0, high = base, size = (1, long)) # En este metodo, base = base - 1
    return l_pol[0]

# Calculo del valor del polinomio usando la regla de Horner
# Ej, la lista [4 7 6] nos da polinomio f(x) = 6x^2 + 7^x + 4 = x*(6x + 7) + 4; si x = 10; f(x) = 674
def poli_2_num(l_pol, base = 10):
    if len(l_pol) == 1:    
        return l_pol[0]
    else:
        return l_pol[0] + base * poli_2_num(l_pol[1:], base)
    
# Genera un numero
def rand_numero(num_digits, base = 10):
    num = 0
    coef_min = -1
    l_pol_max = [base - 1]
    
    # Encontramos el polinomio en el cual se encuentra el numero con el numero de digitos deseado
    # Para ello establecemos que tiene que tiene que satisfacer que tenga longitud + 1 que num_digits
    # porque tanto el numero minimo como maximo se encontrara en dicho polinomio de dicha longitud
    while len(str(num)) < num_digits + 1:
        l_pol_max.insert(0, base - 1)
        num = poli_2_num(l_pol_max, base)
        # print(l_pol_max, num)
    
    num = 0
    # Encontramos el polinomio aleatorio al cual sustituyendole la base nos da el numero que tiene
    # de longitud num_digits
    while True:
        l_pol = rand_polinomio(len(l_pol_max), base)
        num = poli_2_num(l_pol, base)
        if len(str(num)) == num_digits:
            break

    return num

def num_2_pol(num, base = 10):
    # Encontramos el maximo coeficiente del polinomio
    max_deg = math.floor(math.log(num, base))
    l_pol = [0] * (max_deg + 1) # Inicializacion de la lista resultado
    # print("Maximo coeficiente del polinomio: ", max_deg, " Lista polinomica inicial: ", l_pol)

    while num > 0:
        for coef in range(9, -1, -1): # Buscamos el coeficiente maximo
            num_aux = num
            # print("num: ", num, " - num_aux: ", num_aux)
            n = coef*(base**max_deg) # Operacion coef*x^grado polinomico

            if n <= num_aux:
                num_aux -= n
                num = num_aux
                l_pol[max_deg] = coef
                max_deg -= 1
                break

        # print("Lista poli intermedia: ", l_pol)

    print("Lista polinomica: ", l_pol)
    return l_pol

In [7]:
print(rand_numero(10, base = 10))

1541306331


In [13]:
# Multiplicación de polinomios usando el método tradicional
def mult_polinomios(l_pol_1, l_pol_2):
    l_mult = [0] * (len(l_pol_1) + len(l_pol_2) - 1)
    
    for i in range(len(l_pol_1)):
        for j in range(len(l_pol_2)):
            l_mult[i + j] = (l_pol_1[i] * l_pol_2[j]) + l_mult[i + j]
    
    return l_mult

# Multiplicacion de polinomios usando fft
def mult_polinomios_fft(l_pol_1, l_pol_2, fft_func = fft):
    
    # Comprobacion de las longitudes de las listas polinomicas
    if len(l_pol_1) < len(l_pol_2):
        l_pol_1 += [0]*(len(l_pol_2) - len(l_pol_1))
    elif len(l_pol_1) > len(l_pol_2):
        l_pol_2 += [0]*(len(l_pol_1) - len(l_pol_2))
    
    # Correccion de las longitudes de las listas polinomicas para la fft
    k = len(l_pol_1)
    
    if k == 1 :
        return t
    
    pow2 = math.ceil(math.log(k,2))
    
    l_pol_1 = np.array(list(l_pol_1) + (2**pow2 - k) * [float(0)], dtype = complex)
    l_pol_2 = np.array(list(l_pol_2) + (2**pow2 - k) * [float(0)], dtype = complex)
    
    # FFT de cada polinomio
    l_pol_1 = fft_func(l_pol_1)
    l_pol_2 = fft_func(l_pol_2)

    print(l_pol_1, l_pol_2)
    # Inicializacion de la lista donde ira la multiplicacion polinomica
    l_mult = np.array([0] * len(l_pol_1), dtype = complex)
    
    # Multiplicacion polinomica
    for i in range(len(l_pol_1)):
        l_mult[i] = l_pol_1[i] * l_pol_2[i]
        print(l_mult)
    print(l_mult)
        
    # Conjugados, cambio de signo en el termino complejo
    for i in range(len(l_mult)):
        if l_mult[i].imag != 0.0:
            l_mult[i] = np.conjugate(l_mult[i])
    print("Conjugado: ", l_mult)
    # Comprobacion de que la lista no contiene componente imaginaria
    # Si la tuviera, ejecuta la fft hasta que no
    while ([0.0]*len(l_mult) == [elem.imag for elem in l_mult]) == False:
        l_mult = fft_func(l_mult)

    return [int(elem.real) for elem in l_mult]


# Multiplicacion de numeros llevando dichos numeros a sus 
# expresiones polinomicas y siguiendo el metodo tradicional
def mult_numeros(num1, num2):
    # Convertimos los numeros a polinomios con ¿base = 10?
    l_pol_1 = num_2_pol(num1)
    l_pol_2 = num_2_pol(num2)
    
    # Multiplicacion de polinomios
    l_mult = mult_polinomios(l_pol_1, l_pol_2)
    
    # Computo del numero resultante
    num_res = 0
    base = 10         # BASE = 10 SIEMPRE?????????????
    
    for num in l_mult:
        num_res += num*(base**l_mult.index(num))

    return num_res

# Multiplicacion de numeros usando fft
def mult_numeros_fft(num1, num2, fft_func = fft):
    base = 10
    num = 0
    
    # Conversion de los numeros a polinomios, para ejecutar la multiplicacion usando fft
    l_pol_1 = num_2_pol(num1)
    l_pol_2 = num_2_pol(num2)
    # Multiplicacion de los polinomios usando fft
    l_mult = mult_polinomios_fft(l_pol_1, l_pol_2, fft_func)
    
    # print(l_mult)
    
    # Computo del numero
    for i in range(len(l_mult)):
        num += l_mult[i].real*base**i
        
    return num

In [19]:
# l_pol = rand_polinomio(long = 3, base = 10)
# print(l_pol)

# print(poli_2_num(l_pol, base = 10))

# print(num_2_pol(24, base = 10))
# print(num_2_pol(13, base = 10))

# print(mult_numeros(3521, 4217))

print(mult_polinomios_fft([1, 2, 1], [2,1], fft_func = np.fft.fft))
print(mult_numeros_fft(121, 12, fft_func = np.fft.fft))

[4.+0.j 0.-2.j 0.+0.j 0.+2.j] [3.+0.j 2.-1.j 1.+0.j 2.+1.j]
[12.+0.j  0.+0.j  0.+0.j  0.+0.j]
[12.+0.j -2.-4.j  0.+0.j  0.+0.j]
[12.+0.j -2.-4.j  0.+0.j  0.+0.j]
[12.+0.j -2.-4.j  0.+0.j -2.+4.j]
[12.+0.j -2.-4.j  0.+0.j -2.+4.j]
Conjugado:  [12.+0.j -2.+4.j  0.+0.j -2.-4.j]
[8, 20, 16, 4]
Lista polinomica:  [1, 2, 1]
Lista polinomica:  [2, 1]
[4.+0.j 0.-2.j 0.+0.j 0.+2.j] [3.+0.j 2.-1.j 1.+0.j 2.+1.j]
[12.+0.j  0.+0.j  0.+0.j  0.+0.j]
[12.+0.j -2.-4.j  0.+0.j  0.+0.j]
[12.+0.j -2.-4.j  0.+0.j  0.+0.j]
[12.+0.j -2.-4.j  0.+0.j -2.+4.j]
[12.+0.j -2.-4.j  0.+0.j -2.+4.j]
Conjugado:  [12.+0.j -2.+4.j  0.+0.j -2.-4.j]
5808


In [20]:
# 1-C. Midiendo Tiempos

# que genere n_pairs parejas de numeros n1, n2 con num_digits_ini + k*step digitos 
# y devuelva una lista con los tiempos
# medios de sus multiplicacciones n1*n2 de acuerdo al algoritmo habitual
def time_mult_numeros(n_pairs, num_digits_ini, num_digits_fin, step):
    
    l_time = []
    num_digits = num_digits_ini
    k = 0
    
    while k != n_pairs:
    # while num_digits < num_digits_fin:
        time_ini = time.clock() 
        num1 = rand_numero(num_digits)
        num2 = rand_numero(num_digits)
        
        mult = mult_numeros(num1, num2)
        
        # print("Num digits: ", num_digits, " K: ", k, " Step: ", step)
        # print("Nums: ", num1, num2, " Mult: ", mult)
        
        k += 1
        
        if num_digits > num_digits_fin:
            num_digits = num_digits_fin
        else:
            num_digits += k*step
            
        time_fin = time.clock()
        l_time.append(time_fin - time_ini)
        # print(k, n_pairs)
    print(l_time)
    return l_time
    
    # que genere n_pairs parejas de numeros con ´ num_digits_ini + k*step d´ıgitos y devuelva una lista con los tiempos medios
    # de sus multiplicacciones de acuerdo al algoritmo basado en una implementacion (por ejemplo, la nuestra o la de Numpy) ´
    # de la FFT.
def time_mult_numeros_fft(n_pairs, num_digits_ini, num_digits_fin, step, fft_func=fft):
    l_time = []
    num_digits = num_digits_ini
    k = 0
    
    while k != n_pairs:
    # while num_digits < num_digits_fin:
        time_ini = time.clock()
        num1 = rand_numero(num_digits)
        num2 = rand_numero(num_digits)
        
        mult = mult_numeros_fft(num1, num2)
        
        print("Num digits: ", num_digits, " K: ", k, " Step: ", step)
        print("Nums: ", num1, num2, " Mult: ", mult)
        
        k += 1
        
        if num_digits > num_digits_fin:
            num_digits = num_digits_fin
        else:
            num_digits += k*step
            
        time_fin = time.clock()
        l_time.append(time_fin - time_ini)
        # print(k, n_pairs)
    print(l_time)
    return l_time
    

In [ ]:
#time_mult_numeros(10, 2, 4, 1)
time_mult_numeros_fft(2, 2, 4, 1)

Lista polinomica:  [9, 3]
Lista polinomica:  [4, 2]
[12.+0.j  3.+0.j] [6.+0.j 2.+0.j]
[72.+0.j  0.+0.j]
[72.+0.j  6.+0.j]
[72.+0.j  6.+0.j]
Conjugado:  [72.+0.j  6.+0.j]
Num digits:  2  K:  0  Step:  1
Nums:  39 24  Mult:  132
Lista polinomica:  [6, 1, 9]
Lista polinomica:  [3, 0, 7]
[16.+0.0000000e+00j  9.+0.0000000e+00j 14.+1.2246468e-16j
  0.+0.0000000e+00j] [10.+0.j  7.+0.j 10.+0.j  0.+0.j]
[160.+0.j   0.+0.j   0.+0.j   0.+0.j]
[160.+0.j  63.+0.j   0.+0.j   0.+0.j]
[160.+0.0000000e+00j  63.+0.0000000e+00j 140.+1.2246468e-15j
   0.+0.0000000e+00j]
[160.+0.0000000e+00j  63.+0.0000000e+00j 140.+1.2246468e-15j
   0.+0.0000000e+00j]
[160.+0.0000000e+00j  63.+0.0000000e+00j 140.+1.2246468e-15j
   0.+0.0000000e+00j]
Conjugado:  [160.+0.0000000e+00j  63.+0.0000000e+00j 140.-1.2246468e-15j
   0.+0.0000000e+00j]


/home/alex/.virtualenvs/dalg/lib/python3.5/site-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in cdouble_scalars
/home/alex/.virtualenvs/dalg/lib/python3.5/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in cdouble_scalars


In [263]:
l = [12.+0.j, -2.-4.j,  0.+0.j, -2.+4.j]
fft1 = np.fft.fft(l)
print(fft1)
print(np.fft.ifft(fft1))

[ 8.+0.j  4.+0.j 16.+0.j 20.+0.j]
[12.+0.j -2.-4.j  0.+0.j -2.+4.j]
